In [2]:
import torch
import cv2
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
img1=cv2.imread("reference.jpg")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
print(np.shape(img1))
VAL_RATIO = 0.2
#The ratio of the total training set used only for validation

(1080, 1920, 3)


In [ ]:
dataset = torchvision.datasets.LFWPeople("data",split="train", download=True, transform=transforms.ToTensor())

INPUT_SHAPE = (3,250,250)
VAL_RATIO = 0.2
val_len = int(len(dataset) * VAL_RATIO)
train_len = len(dataset) - val_len

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])

print(val_len,train_len)


image, label = train_dataset[0]
image, label

Files already downloaded and verified
1905 7620


(tensor([[[0.8392, 0.8431, 0.8706,  ..., 0.0157, 0.0314, 0.0157],
          [0.8431, 0.8431, 0.8627,  ..., 0.5804, 0.3529, 0.2235],
          [0.8471, 0.8431, 0.8510,  ..., 0.3490, 0.2627, 0.1725],
          ...,
          [0.6627, 0.6784, 0.6902,  ..., 0.9294, 0.9490, 0.9529],
          [0.6667, 0.6706, 0.6745,  ..., 0.8941, 0.9176, 0.9137],
          [0.6549, 0.6627, 0.6627,  ..., 0.8902, 0.8941, 0.8902]],
 
         [[0.6941, 0.6941, 0.7098,  ..., 0.0000, 0.0157, 0.0000],
          [0.6980, 0.6941, 0.7020,  ..., 0.5686, 0.3373, 0.2039],
          [0.7020, 0.6941, 0.7020,  ..., 0.3255, 0.2471, 0.1569],
          ...,
          [0.5255, 0.5294, 0.5412,  ..., 0.9137, 0.9333, 0.9451],
          [0.5333, 0.5373, 0.5412,  ..., 0.8824, 0.9059, 0.9059],
          [0.5373, 0.5451, 0.5451,  ..., 0.8784, 0.8824, 0.8863]],
 
         [[0.6196, 0.6196, 0.6392,  ..., 0.0000, 0.0000, 0.0000],
          [0.6235, 0.6196, 0.6314,  ..., 0.5098, 0.3020, 0.1804],
          [0.6275, 0.6196, 0.6275,  ...,

In [ ]:
image.shape
len(train_dataset), len(train_dataset), len(val_dataset), len(val_dataset)

(7620, 7620, 1905, 1905)

In [ ]:
image, label = train_dataset[0]
print(f"Image shape: {image.shape}")
transform = transforms.ToPILImage()
img = transform(image)
img.show()

Image shape: torch.Size([3, 250, 250])


In [ ]:
BATCH_SIZE = 127